function additions

split the data into training and testing sets

fit an ARMA model to the training data

evaluate the model on the testing data

plot the actual vs predicted values

predict future values using the ARMA model

place limit order to buy GBP/USD at a specific price based on ARMA model prediction


write python code for me to predict gbp to usd by connecting to interactive broker api. I want you to:
split the data into training and testing sets

fit an ARIMA model to the training data

evaluate the model on the testing data

plot the actual vs predicted values

predict future values using the ARIMA model

place limit order to buy GBP/USD at a specific price based on ARMA model prediction

In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.order_state import OrderState
from ibapi.execution import Execution
from ibapi.common import OrderId

import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import numpy as np

In [2]:
class TestApp(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.predicted_price = None
        self.nextOrderId = None
        self.bars = []

    def nextValidId(self, orderId: OrderId):
        self.nextOrderId = orderId
        print(f"Next valid order ID: {orderId}")

        # Define GBP/USD contract
        mycontract = Contract()
        mycontract.symbol = "GBP"
        mycontract.secType = "CASH"
        mycontract.exchange = "IDEALPRO"
        mycontract.currency = "USD"

        # Request historical data
        self.reqHistoricalData(
            reqId=1,
            contract=mycontract,
            endDateTime="",
            durationStr="30 D",
            barSizeSetting="1 hour",
            whatToShow="MIDPOINT",
            useRTH=0,
            formatDate=1,
            keepUpToDate=False,
            chartOptions=[]
        )

    def historicalData(self, reqId, bar):
        self.bars.append((bar.date, bar.close))

    def historicalDataEnd(self, reqId, start, end):
        df = pd.DataFrame(self.bars, columns=["date", "close"])
        df["close"] = df["close"].astype(float)
        df.set_index(pd.to_datetime(df["date"]), inplace=True)
        df = df["close"]

        # Train/test split
        split_idx = int(len(df) * 0.8)
        train, test = df[:split_idx], df[split_idx:]

        # Fit ARIMA
        model = ARIMA(train, order=(1, 1, 1))
        model_fit = model.fit()

        # Evaluate
        predictions = model_fit.forecast(steps=len(test))
        mse = ((predictions.values - test.values) ** 2).mean()
        print(f"Test MSE: {mse:.6f}")

        # Predict future 1-step
        forecast = model_fit.forecast(steps=1)
        self.predicted_price = float(forecast.iloc[0])
        print(f"Predicted next GBP/USD: {self.predicted_price}")

        # Request contract details to place order
        mycontract = Contract()
        mycontract.symbol = "GBP"
        mycontract.secType = "CASH"
        mycontract.exchange = "IDEALPRO"
        mycontract.currency = "USD"

        self.reqContractDetails(2, mycontract)

    def contractDetails(self, reqId: int, contractDetails):
        print("Contract details received. Placing order...")

        myorder = Order()
        myorder.orderId = self.nextOrderId
        myorder.action = "BUY"
        myorder.orderType = "LMT"
        myorder.tif = "GTC"
        myorder.lmtPrice = round(self.predicted_price - 0.0001, 5)
        myorder.totalQuantity = float(10000)  # make float for fractional size rules
        myorder.eTradeOnly = False
        myorder.firmQuoteOnly = False

        self.placeOrder(myorder.orderId, contractDetails.contract, myorder)

    def openOrder(self, orderId: OrderId, contract: Contract, order: Order, orderState: OrderState):
        print(f"openOrder: {orderId}, contract: {contract.symbol}, price: {order.lmtPrice}")

    def execDetails(self, reqId: int, contract: Contract, execution: Execution):
        print(f"Execution details: {execution}")

app = TestApp()
app.connect("127.0.0.1", 7497, clientId=200)
app.run()


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil
ERROR 1 10285 Your API version does not support fractional size rules. Please upgrade to a minimum version 163.


Next valid order ID: 2


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe228893760>>
Traceback (most recent call last):
  File "/Users/snowyan/PycharmProjects/JupyterProject/.venv/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


AttributeError: 'tuple' object has no attribute 'tb_frame'